In [ ]:
import os

os.environ['HF_HOME'] = '/root/ramdisk'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from transformers import GptOssForCausalLM, AutoModelForCausalLM, AutoTokenizer, Mxfp4Config
from peft import PeftModel
import torch

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

In [ ]:
class Model(GptOssForCausalLM):
    def __init__(self, config):
        super().__init__(config)

In [ ]:
quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="kernels-community/vllm-flash-attn3",
    torch_dtype=torch.bfloat16, 
    use_cache=False, 
    # quantization_config=quantization_config,
)
model = Model.from_pretrained("openai/gpt-oss-120b", **model_kwargs).cuda()

In [ ]:
!ls malaysian-reasoning-20b

In [ ]:
peft_model_id = "malaysian-reasoning-20b/checkpoint-861"
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

In [ ]:
q = """
Budak itu sangat nakal, pantang orang leka sedikit, duit syiling pun dikebasnya.

terjemah ke kedah
"""

system = 'First, you try to think step-by-step in {{lang}}, after that, put your final answer within $\\boxed{}$.'
messages = [
    {"role": "system", "content": system.replace('{{lang}}', 'malay')},
    {"role": "user", "content": q},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

In [ ]:
gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 1.0, "top_p": None, "top_k": None}

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]

In [ ]:
response

In [ ]:
tokenizer.save_pretrained('gpt-oss-20b-Malaysian-Reasoning-SFT-v0.1')

In [ ]:
model.save_pretrained('gpt-oss-20b-Malaysian-Reasoning-SFT-v0.1')

In [ ]:
# !hf upload Scicom-intl/gpt-oss-20b-Malaysian-Reasoning-SFT-v0.1 \
#     gpt-oss-20b-Malaysian-Reasoning-SFT-v0.1

In [ ]:
from transformers import AutoConfig

quantization_config = Mxfp4Config(dequantize=True)

config = AutoConfig.from_pretrained(
    "openai/gpt-oss-20b",
    attn_implementation="kernels-community/vllm-flash-attn3",
    torch_dtype=torch.bfloat16,
    use_cache=False,
    quantization_config=quantization_config,
)

In [ ]:
config.quantization_config.__dict__